In [ ]:
pip install folium

pip install haversine



In [ ]:

import xml.etree.ElementTree as ET
import pprint
import random as rd
import folium as flMap
from haversine import haversine
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import statistics



In [ ]:


'''Classes Definitions

- UserObject : represents individual user object
- Latlon : represent object of Lat Lon data and its methods
- WSSCR : represent WSSCR region object
- SurveillanceCode : represents object which will convert a existing GPS info to some needed object for some process

'''


class UserObject:
    def __init__(self,id,lat_lon_obj,wsscr=None):
        self.id = id
        self.lat_lon_obj = lat_lon_obj
        self.wsscr=wsscr
        self.Positions = [lat_lon_obj]

    def addTrackPosition(self,pos):
        self.Positions.append(pos)

    def getTrackPosition(self):
        return self.Positions


class LatLon:
    def __init__(self,lat,lon):
        self.lat = float(lat)
        self.lon = float(lon)

    def getLatLonAsList(self):
        return [self.lat,self.lon]

    def getLatLonAsTuple(self):
        return (self.lat,self.lon)


class SurveillanceCode:
    WD = ["SUN","MON","TUE","WED","THU","FRI","SAT"]
    TS = [("T1","T3"),("T4","T6"),("T7","T9"),("T10","T12"),("T13","T15"),("T16","T18"),("T19","T21"),("T22","T24")]
    TM = ["flying","staying","vehicle","walking"]


    def __init__(self,userid,tm,wd,ts,lat_lon):
        self.userid = userid
        self.tm = tm
        self.wd = wd
        self.ts = ts
        self.lat_lon = lat_lon

        self.GeneratePositionIdentifier()
        

    def GeneratePositionIdentifier(self):
        entropy = rd.randint(100,999)
        self.id = self.userid.split()[0] + self.tm[:2] + self.wd[:2] + self.ts[0] + str(entropy)


class WSSCR:
    k_anonimity_threshold = 5
    cloaked_search_space_radius= 1  # kilo meters
    def __init__(self,WSSCR_id,WSSCR_list):
        self.id = WSSCR_id
        self.wsscr_users = WSSCR_list      # userlists
    
    def center_of_wsscr(self):
        self.wsscr_center = None
        if(len(self.wsscr_users)>0):
            lat_mean = statistics.mean([ user.lat_lon_obj.lat for user in self.wsscr_users])
            lon_mean = statistics.mean([ user.lat_lon_obj.lon for user in self.wsscr_users])

            return (lat_mean,lon_mean)
            



In [ ]:
'''Parsing xml file for lat lon, 

- where all lat lon can be treated as individual user existing or active to the nearest MEC network
- In Actual implementation, this can be replaced with a real time system which gathers user's real time GPS information 

'''

tree = ET.parse('route_2.gpx')
root = tree.getroot()

#print(root.tag)

lat_lon_list=[]
for i in root.iter("wpt"):
    lat_lon_list.append(LatLon(i.attrib['lat'],i.attrib['lon']))


#pprint.PrettyPrinter().pprint(lat_lon_list)

In [ ]:


def userIdGenerator():
    id=rd.randint(100,1000)
    return "user" + str(id)

def addWSSCR(user,userList):
    t_diff=0
    radius = WSSCR.cloaked_search_space_radius
    min = float('inf')
    for i in userList:
        diff = round(haversine((user.lat_lon_obj.lat,user.lat_lon_obj.lon),(i.lat_lon_obj.lat,i.lat_lon_obj.lon)),3)
        t_diff=diff
        #print(diff)
        if(diff<radius):
            if(diff<min):
                if(i.wsscr):
                    user.wsscr = i.wsscr
    if(user.wsscr==None):
        user.wsscr = "WSSCR-Region:"+ str(rd.randint(100,1000))
    
    #print(t_diff,user.wsscr)


userList = []
temp_lat_lon_list = lat_lon_list.copy()
for i in lat_lon_list:
    userLatLon =  rd.choice(temp_lat_lon_list)
    user = UserObject(userIdGenerator(),userLatLon)
    userList.append(user)    
    temp_lat_lon_list.remove(i)

for i in userList:
    addWSSCR(i,userList)




In [ ]:
wsscr_list=[]

for i in userList:
    if(i.wsscr in wsscr_list):
        continue
    wsscr_list.append(i.wsscr)

wsscr_dict = {}

for i in wsscr_list:
    #count = 0
    wsscr_dict[i]=[]
    for j in userList:
        if(j.wsscr == i):
            #count += 1 
            wsscr_dict[i].append(j)

# pprint.PrettyPrinter().pprint(wsscr_dict)

wsscr_objects_list =[]

for x in wsscr_dict:
    wsscr= WSSCR(x,wsscr_dict[x])
    wsscr_objects_list.append(wsscr)


#pprint.PrettyPrinter().pprint(wsscr_objects_list)


    

In [ ]:
map = flMap.Map(location=[lat_lon_list[0].lat,lat_lon_list[0].lon], zoom_start=14, tiles="Stamen Terrain")

for wsscr in wsscr_objects_list:
    if(len(wsscr.wsscr_users)>0):
        for user in wsscr.wsscr_users:
                flMap.Marker([user.lat_lon_obj.lat,user.lat_lon_obj.lon],popup=f"{user.wsscr}, \n{user.lat_lon_obj.lat},{user.lat_lon_obj.lon}",tooltip=f"<i>{user.id}</i>").add_to(map)
    flMap.Circle(
        location=list(wsscr.center_of_wsscr()),
        radius=1000*WSSCR.cloaked_search_space_radius + 500,
        popup=wsscr.id,
        color="#3186cc",
        fill=True,
        fill_color="#3186cc",
        ).add_to(map)




''' A real time PBS user object for some PBS service

- this can be replaced with a PBS device sending its GPS info to some anonmization server for PBS confidentiality

'''
u1 = UserObject("testing User",LatLon(30.87374,75.81251))

#showing map

map



In [ ]:
''' function to generate Random paths

- this can be replaced by real time GPS server ( an anonmization server), which will responsible for  Extraction of all neighbor users GPS path info

'''


def PathGeneration(user=None,val=0.0012):
    if(user):
        entropy = val

        lat = user.lat_lon_obj.lat + entropy
        lon = user.lat_lon_obj.lon + entropy
        user.addTrackPosition(LatLon(lat,lon))
        entropy +=  entropy
    else:
        for user in userList:
            entropy = val
            lat = user.lat_lon_obj.lat + entropy
            lon = user.lat_lon_obj.lon + entropy
            user.addTrackPosition(LatLon(lat,lon))

            # flMap.Marker([lat,lon],
            #             color=color,
            #             icon=flMap.Icon(color=color, icon='info-sign'),
            #             popup=f"{user.wsscr}, {lat},{lon}",
            #             tooltip=f"<i>{user.id}</i>").add_to(map)

    entropy +=  entropy


''' some testing part function

- Verification of generated User's path by showing on map

'''

def VerificationOfTrack():
     for user in userList:
        user_tracks =user.getTrackPosition()
        color_list = ["red","blue","green","black"]
        if(len(user_tracks)>0):
            count = 0        
            for latlon in user_tracks:        
                color = color_list[count%len(color_list)]
                lat = latlon.lat 
                lon = latlon.lon
                flMap.Marker([lat,lon],
                            color=color,
                            icon=flMap.Icon(color=color, icon='info-sign'),
                            popup=f"{user.wsscr}, {lat},{lon}",
                            tooltip=f"<i>{user.id}</i>").add_to(map)
                count += 1



'''Path Generation Process

- this method can be replaced by real time GPS server api data ( an anonmization server), which will responsible for  Extraction of all neighbor users GPS path info
    as well as PBS User

'''
#path generation of all users near to MEC

PathGeneration(val = 0.0022)
PathGeneration(val = 0.0033)
PathGeneration(val = 0.0044)
PathGeneration(val = 0.00567)
PathGeneration(val = 0.00667)
#VerificationOfTrack()

'''
- path generation of PBS user 
- prev history path of PBS user

'''

PathGeneration(u1,val = 0.0022)
PathGeneration(u1,val = 0.0033)
PathGeneration(u1,val = 0.0044)
PathGeneration(u1,val = 0.00567)
PathGeneration(u1,val = 0.00667)




In [ ]:
''' KNN Algorithm

- This classfies the, WSSCR region of PBS user with its neighbor active users

'''

#implementing knn algorithm
  
# Loading data
temp_list = [[user.id,user.lat_lon_obj.lat,user.lat_lon_obj.lon,user.wsscr] for user in userList]
dataset = pd.DataFrame(temp_list,columns=["User ID","Lat","Lon","WSSCR ID"])

#print(dataset)

# # Create feature and target arrays
x = dataset.iloc[:,1:3].values
y = dataset.iloc[:,-1].values
  
# Split into training and test set
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.1)


knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)


In [ ]:
# print(knn.predict(X_test))
#y_pred = knn.predict(X_test)
#print(y_pred)

#print(classification_report(y_test,y_pred))
#print(confusion_matrix(y_test,y_pred))


In [ ]:
data =[[u1.lat_lon_obj.lat,u1.lat_lon_obj.lon]]

predicted_wsscr = knn.predict(data)
u1.wsscr= predicted_wsscr
print(u1.wsscr)

print(wsscr_list)
flMap.Marker([u1.lat_lon_obj.lat,u1.lat_lon_obj.lon],
                color='green',
                icon=flMap.Icon(color='green', icon='info-sign'),
                popup=f"{u1.wsscr}, {u1.lat_lon_obj.lat},{u1.lat_lon_obj.lon}",
                tooltip=f"<i>{u1.id}</i>").add_to(map)


map

In [ ]:
'''
Result of Position Recognition phase is : set of position of all users of the resulted WSSCR 

---> output:> u1 (PBS user object ) ( having id, positions, WSSCR)

'''

'''
SurveillanceCode Sequence Generation of PBS user

-some assumptions are:

    user's TM: walking
    user's WD: SUN

- this can be avoided while working with real time system data

'''

def SurveillanceCodeSequenceGeneration(user):
    SurveillanceCode_list=[]
    ind = 0
    for pos in user.getTrackPosition():
        ind += 1
        SurveillanceCode_list.append(SurveillanceCode(user.id,SurveillanceCode.TM[3],SurveillanceCode.WD[0],SurveillanceCode.TS[ind],pos))
    return SurveillanceCode_list

SurveillanceCode_list = SurveillanceCodeSequenceGeneration(u1)


'''
Function to Generate some Random sequential track 

- this can also be replaced by real time user' track position GPS data

'''

def ExperimentalTrackSequenceGeneration(sur_list,l_num):
    TSL =[]
    for i in range(l_num):
        randomness = rd.randint(1,10)
        #print(randomness)
        TrackSequence = sur_list.copy()
        for i in range(randomness):
            rd.shuffle(TrackSequence)
        TSL.extend(TrackSequence)

    return TSL


Psurveillancecode = ExperimentalTrackSequenceGeneration(SurveillanceCode_list,3)
#pprint.PrettyPrinter().pprint(ExperimentalTrackSequenceList)




In [ ]:
'''
HMM machine learning algo implementation

- for predicting PBS user position in its WSSCR region to amentisized them with PBS

- HMM Position Prediction of PBS user on the basis of Survellance code

'''

def generate_markov_model_(sur_list, n_gram=2):
    markov_model = {}
    for i in range(len(sur_list)-n_gram-1):
        curr_state, next_state = 0, 0
        for j in range(n_gram):
            curr_state = sur_list[i+j]
            next_state = sur_list[i+j+n_gram]

        if curr_state not in markov_model:
            markov_model[curr_state] = {}
            markov_model[curr_state][next_state] = 1
        else:
            if next_state in markov_model[curr_state]:
                markov_model[curr_state][next_state] += 1
            else:
                markov_model[curr_state][next_state] = 1
    
    # calculating transition probabilities
    for curr_state, transition in markov_model.items():
        total = sum(transition.values())
        for state, count in transition.items():
            markov_model[curr_state][state] = count/total
        
    return markov_model

# this data can be get real time using intermediate servers

dataset = [ "+".join([str(t) for t in pos.lat_lon.getLatLonAsList()]) for pos in Psurveillancecode]



Markov_Model_obj = generate_markov_model_(dataset)

#pprint.PrettyPrinter().pprint(Markov_Model_obj)

# t = ["+".join([str(t) for t in pos.lat_lon.getLatLonAsList()]) for pos in ExperimentalTrackSequenceList]
# pprint.PrettyPrinter().pprint(t)


def PredictPosition(markov_model, pos):
    curr_state = pos
    for i in range(len(u1.getTrackPosition())):
        next_state = rd.choices(list(markov_model[curr_state].keys()),
                                        list(markov_model[curr_state].values()))[0]      
        curr_state = next_state

    predict_state = curr_state.split("+")
    return predict_state


user_random_pos = "+".join(str(x) for x in rd.choices(u1.getTrackPosition(),k=1)[0].getLatLonAsList())   #"+".join(str(x) for x in pos.getLatLonAsList())

PredictedPosition = PredictPosition(Markov_Model_obj,user_random_pos)

print(PredictedPosition)




In [ ]:
flMap.Marker(PredictedPosition,
                color='red',
                icon=flMap.Icon(color='red', icon='info-sign'),
                popup=f"{u1.wsscr}, {u1.lat_lon_obj.lat},{u1.lat_lon_obj.lon}",
                tooltip=f"<i>{u1.id}</i><br><b> Predicted Position </b>").add_to(map)

map